In [1]:
# Name: example_analysis_data_creation.ipynb
# uthors: Stephan Meighen-Berger
# Example how to construct a data set to use for later analyses

In [2]:
# General imports
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import pickle

In [3]:
# Adding path to module
sys.path.append("../")

In [4]:
# picture path
PICS = '../pics/'

In [5]:
# Module imports
from fourth_day import Fourth_Day, config

In [6]:
# Scenario Settings
# These are general settings pertaining to the simulation run
config['scenario']['population size'] = 10  # The starting population size
config['scenario']['duration'] = 600 * 5  # Total simulation time in seconds
config['scenario']['exclusion'] = True  # If an exclusion zone should be used (the detector)
config['scenario']['injection']['rate'] = 1e-2  #  Injection rate in per second, a distribution is constructed from this value
config['scenario']['injection']['y range'] = [5., 10.]  # The y-range of injection
config['scenario']['light prop'] = {  # Where the emitted light should be propagated to (typically the detector location)
            "switch": True,  # If light should be propagated
            "x_pos": 3.,  # The x-coordinates
            "y_pos": 0.5 * 15. - 0.15,  # The y-coordinates
        }
config['scenario']['detector'] = {  # detector specific properties, positions are defined as offsets from the light prop values
    "switch": True,  # If the detector should be modelled
    "type": "PMTSpec",  # Detector name, implemented types are given in the config
    "response": True,  # If a detector response should be used
    "acceptance": "Flat",  # Flat acceptance
    "mean detection prob": 0.5  # Used for the acceptance calculation
}
# ---------------------------------------------
# Organisms
# Organisms properties are defined here
config['organisms']['emission fraction'] = 0.2  # Amount of energy an organism uses per pulse
config['organisms']['alpha'] = 1e1  # Proportionality factor for the emission probability
config['organisms']["minimal shear stress"] = 0.05  # The minimal amount of shear stress needed to emit (generic units)
config["organisms"]["filter"] = 'depth'  # Method of filtering organisms (here depth)
config["organisms"]["depth filter"] = 1000.  # Organisms need to exist below this depth
# ---------------------------------------------
# Geometry
# These settings define the geometry of the system
# Typically a box (simulation volume) with a spherical exclusion zone (detector)
config['geometry']['volume'] = {
    'function': 'rectangle',
    'x_length': 30.,
    'y_length': 15.,
    'offset': None,
}
# Reduce the observation size to reduce the computational load
config['geometry']['observation'] = {
    'function': 'rectangle',
    'x_length': 30.,
    'y_length': 15.,
    "offset": np.array([0., 0.]),
}
# The detector volume
config['geometry']["exclusion"] = {
    "function": "sphere",
    "radius": 0.15,
    "x_pos": 3.,
    "y_pos": 0.5 * 15. - 0.15,
}
# ---------------------------------------------
# Water
# Properties of the current model
config['water']['model']['name'] = 'custom'  # Use a custom (non analytic) model
config['water']['model']['off set'] = np.array([0., 0.])  # Offset of the custom model
config['water']['model']['directory'] = "../data/current/Parabola_5mm/run_2cm_npy/"  # The files used by the custom model
config['water']['model']['time step'] = 5.  # in Seconds

In [7]:
# Which detectors we want to use
wavelengths = {
    "Detector 1": ["1", "#4575b4"],
    "Detector 5": ["2", "#91bfdb"],
    "Detector 8": ["3", "#e0f3f8"],
    "Detector 3": ["4", "#fee090"],
    "Detector 10": ["5", "#fc8d59"],
}

In [8]:
# Launching multiple simulations to use in the analysis
seeds = np.arange(100)[1:]
print(seeds)
counter = 0
for seed in tqdm(seeds):
    # General
    config["general"]["random state seed"] = seed
    # Creating a fourth_day object
    fd = Fourth_Day()
    # Launching solver
    fd.sim()
    # Fetching relevant data
    # Totals
    total = fd.measured["Detector 1"].values
    for detector in wavelengths.keys():
        if detector == "Detector 1":
            continue
        fd.measured[detector].values
        total += fd.measured[detector].values
    storage_dic = {
        "time": fd.t,
        "data": total
    }
    pickle.dump(storage_dic, open("../data/storage/vort_run_%d.p" % counter, "wb"))
    counter += 1

  0%|                                                                                           | 0/99 [00:00<?, ?it/s]

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96
 97 98 99]


100%|███████████████████████████████████████████████████████████████████████████████| 99/99 [5:14:22<00:00, 190.53s/it]
